In [44]:
from bs4 import BeautifulSoup
import requests
import re
from tqdm import tqdm
import random
import json
import time
from collections import deque
import os

def notify(title, text):
    os.system("""
              osascript -e 'display notification "{}" with title "{}"'
              """.format(text, title))

USER_AGENT = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.109 Safari/537.36"

In [45]:
def get_repo(i):
    results = []
    search = f"https://github.com/search?p={i}&q=stars%3A%3E2000+language%3APython&type=Repositories"
    response = requests.get(search, headers={'User-Agent': USER_AGENT})
    bs = BeautifulSoup(response.content,'html.parser')
    curr_page_content = bs.select("#js-pjax-container > div > div.col-12.col-md-9.float-left.px-2.pt-3.pt-md-0.codesearch-results > div")[0].find_all("li")
    for c in curr_page_content:
        try:
            title = c.find_all("a", class_="v-align-middle")[0].text
            url = json.loads(c.find_all("a", class_="v-align-middle")[0].get("data-hydro-click"))["payload"]["result"]["url"]
            about = c.find_all("p", class_="mb-1")[0].text.replace("\n", "").replace("  ", "")
            results.append(
                {
                    "title": title,
                    "url": url,
                    "about": about
                }
            )
        except:
            continue
    return results

In [47]:
results = []
fail = deque(maxlen=100)
pbar = tqdm(list(range(1, 101)))
for i in pbar:
    try:
        results += get_repo(i)
    except Exception as e:
        print(f"page {i} failed, push to queue")
        fail.append(i)
    sleep_time = random.randint(8, 12)
    pbar.set_description(f"Sleep {sleep_time} s - ")
    time.sleep(sleep_time)
while len(fail)!=0:
    print(f"refetching page {i}")
    i = fail.popleft()
    try:
        results += get_repo(i)
    except Exception as e:
        print(f"page {i} failed, push to queue")
        fail.append(i)
notify(f"repo data downloaded", f"Total {len(fail)} fail.")

Sleep 12 s - : 100%|██████████| 100/100 [17:35<00:00, 10.55s/it]


In [48]:
import pickle
with open("github_repos.pkl", "wb") as f:
    pickle.dump(results, f)